In [1]:
#import package
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from combo.models.classifier_stacking import Stacking
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np

In [2]:
#load data
data=pd.read_csv("train.csv")
train_X=data.iloc[:,0:-1]
train_y=data.iloc[:,-1]
test_X=pd.read_csv("test.csv")

In [3]:
#data preprocessing
sc=StandardScaler()
ad_train_X=train_X.iloc[:,[3,6,7,8,9,12]]
ad_test_X=test_X.iloc[:,[3,6,7,8,9,12]]
ad1_train_X=sc.fit_transform(ad_train_X)
ad1_test_X=sc.transform(ad_test_X)

ohe=OneHotEncoder()
ad2_train_X=train_X.iloc[:,[4,5,10,11]]
ad2_test_X=test_X.iloc[:,[4,5,10,11]]
ad3_train_X=ohe.fit_transform(ad2_train_X).toarray()
ad3_test_X=ohe.transform(ad2_test_X).toarray()

df1=pd.DataFrame(ad1_train_X)
df2=pd.DataFrame(ad3_train_X)
for i in range(6,15):
    df1[i]=df2[i-6]
df3=pd.DataFrame(ad1_test_X)
df4=pd.DataFrame(ad3_test_X)
for i in range(6,15):
    df3[i]=df4[i-6]

df1.drop(columns=[8,9,11,12,13],inplace=True)
df3.drop(columns=[8,9,11,12,13],inplace=True)

adj_train_X=df1
adj_test_X=df3

In [6]:
classifiers = [XGBClassifier(use_label_encoder=False,max_depth=3,learning_rate=0.27,eval_metric=['logloss','auc','error']),
               XGBClassifier(use_label_encoder=False,max_depth=3,learning_rate=0.11,eval_metric=['logloss','auc','error'])]
clf = Stacking(base_estimators=classifiers) 
clf.fit(adj_train_X,train_y)
pred_y=clf.predict(adj_test_X)

In [7]:
#create csv
outcome={"RowNumber":test_X["RowNumber"],
         "Exited":pred_y}
df_outcome=pd.DataFrame(outcome)
df_outcome.to_csv("predict_74.csv")